***Import Library's***

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline

***Load the data***

In [2]:
print("Proceso de estandarizacion inmuebles de metrocuadrado")
print("- Cargando data set .....")

df_origen = pd.read_csv('../../2.Export/1.Ext/variables_inmuebles_metrocudrado.zip')
n_rows_ori,n_columns_ori =df_origen.shape
print("- Cantidad de registros data set original: ",n_rows_ori)

Proceso de estandarizacion inmuebles de metrocuadrado
- Cargando data set .....
- Cantidad de registros data set original:  30496


C:\Users\RC\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10,13,26,29,36,38,43,45,52,58,60,69,82,84,94) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


***Eliminar duplicados***

In [3]:
print("- Eliminar duplicados")
train = df_origen.drop_duplicates().reset_index(drop=True)
n_rows,n_columns =train.shape
n_row_drop = n_rows_ori - n_rows
print("- Cantidad de registros ahora: ",n_rows," # duplicados: ",n_row_drop)

- Eliminar duplicados
- Cantidad de registros ahora:  30496  # duplicados:  0


***Limpieza de NAN***

In [4]:
data_count_null = train.isnull().sum().reset_index(name='q_nan')

# Variable identificadas como no utiles
columns_to_drop = ['Ancho','Características del Parqueadero','Con muebles','Depósitos','Fondo','Número de Ascensores',
                   'Número de Closets','Número de líneas teléfonicas','Terraza/Balcón','Tipo de Cortinas',
                   'Tipo de calentador','Tipo de estufa','Tipo de parqueadero','Tipo de piso en alcobas',
                   'Tipo de piso en comedor','Tipo de piso en estudio','Tipo de piso en sala','Tipo instalación de gas',
                   'Valor de arriendo','areaPrivada','businessTypeId','cityname','hdCompanyUrl','hdIsOcasional',
                   'hdnBusinessType','nameProperty','nomBarrio','nombreEmpresa','propertyTypeId','valorArriendo',
                   'Área Terraza/Balcón','Área privada','Habitaciones',
                  'Área construida','Área del lote','Baños']

#identifica los campos que tengan el 90% de sus 
for columns in data_count_null[data_count_null['q_nan']==(n_rows)]['index']:
    columns_to_drop.append(columns)

train = train.drop(columns_to_drop,axis='columns')

***Manejo de extras o complementos***

- Extra1: El campo viene con las descripciones de los compementos -- No se debe hacer cambios
- Extra2: Contiene si o no, se debe tomar el nombre del campo como el atributo de complemento
- Extra3: El campo contiene la descripción y dentro de la misma indica que tipo de extra es
- Extra4: El campo contiene la descripción pero es necesario complementar la información con el nombre del campo

In [5]:
train_test = train.copy()

In [6]:
extra2_descripcion = ['Ascensor','Cerca Centros Comerciales','Cerca Colegios / Universidades','Cerca Parques',
                      'Cerca Supermercados','Cerca Transporte Público','Conjunto cerrado','Cuarto de servicio',
                      'En interior y/o bloque','Esquinero','Estudio o biblioteca','Parqueadero cubierto',
                      'Zona de BBQ','Zona de lavanderia']

train_test['extra2'] = ''
for extra2 in extra2_descripcion:
    train_test[extra2] = train_test[extra2].apply(lambda x: extra2+',' if x == 'Si' else '')
    train_test['extra2'] = train_test['extra2']+train_test[extra2]
    #train_test = train_test.drop([extra2],axis='columns')


In [7]:
extra3_descripcion = ['Tipo comedor','Tipo de Casa','Tipo de acabado piso']

train_test['extra3'] = ''
for extra3 in extra3_descripcion:
    train_test[extra3] = train_test[extra3].apply(lambda x: x+',' if x==x else '')
    train_test['extra3'] = train_test['extra3']+train_test[extra3]
    train_test = train_test.drop([extra3],axis='columns')

In [8]:
extra4_descripcion = ['Tipo de Cocina','Vigilancia','Vista']

train_test['extra4'] = ''
for extra4 in extra4_descripcion:
    
    train_test[extra4] = train_test[extra4].apply(lambda x: extra4+' '+str(x)+',' if x==x else ' ')
    train_test['extra4'] = train_test['extra4']+train_test[extra4]
    train_test = train_test.drop([extra4],axis='columns')


In [9]:
train_test['extra'] = train_test['complemento']+train_test['extra2']+train_test['extra3']+train_test['extra4']


In [10]:
train = train_test.drop(['complemento','extra2','extra3','extra4'],axis='columns').copy()

***Homologacion de campos***

In [11]:
homologacion_columns = {'propertyType':'tipo_inmueble',
                        'zona':'zona',
                        'Nombre común del barrio ':'barrio1', 
                        'Nombre del barrio catastral':'sector_catastral', 
                        'description':'descripcion', 
                        'propertyPrice':'precio',
#                         'ContractType':'tipo_contrato',
                        'areaConstruida':'area', 
#                         'Address':'direccion', 
                        'numHabitaciones':'numero_habitaciones', 
                        'numBanos':'num_banos',
                        'Tiempo de construido':'edad',
#                         'Condition':'condicion', 
                        'Número de piso': 'piso_ubicacion', 
                        'Valor de administración':'precio_administracion',
                        'Estrato':'estrato',
                        'numGaraje':'num_garages',
                        'extra':'extras', 
                        'latitude':'latitude',
                        'longitude':'longitude',
                        'Número de niveles':'piso_interior',
                        'Código web':'id_inmueble'}
# Variable identificadas como no utiles
columns_to_drop = []

for column in train.columns:
    if column not in homologacion_columns.keys():
        columns_to_drop.append(column)
        
train = train.drop(columns_to_drop,axis='columns')
train = train.rename(columns = homologacion_columns)

In [12]:
train.to_csv('../../2.Export/2.Final/limpieza_metrocuadrado.csv')
print("Fin limpieza finca raiz")

Fin limpieza finca raiz
